In [1]:
import psycopg2
from datetime import datetime

In [2]:
def polygon_to_geotiff(polygon_coords):
    polygon_wkt = f"LINESTRING({', '.join([f'{lon} {lat}' for lon, lat in polygon_coords])})"
    query = """SELECT ST_AsTIFF(ST_Clip(r.rast, p.geom)) AS cropped_tiff FROM raster_results_1 r, (SELECT ST_SetSRID(ST_MakePolygon(ST_GeomFromText(%s)), 4326) AS geom) p WHERE ST_Intersects(r.rast, p.geom);"""
    
    conn = psycopg2.connect(host="10.223.72.83" ,port="5433", database = "postgres" , user="postgres" , password="punpuntpasswd")
    cursor = conn.cursor()
    
    cursor.execute("SET postgis.gdal_enabled_drivers = 'ENABLE_ALL';")
    cursor.execute(query, (polygon_wkt,))

    result = cursor.fetchone()
    cropped_tiff = result[0] if result else None
    
    out_meta = {'driver': 'GTiff'}
    out_image = cropped_tiff
    
    if cropped_tiff:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"output_{timestamp}.tiff"
        with open(f'/home/kea-trainee-punpun/Week2/polygon_tiff_output/{filename}', 'wb') as f:
            f.write(out_image.tobytes())
        status = f"File '{filename}' created."
    else:
        status = "No cropped raster data found."

    cursor.close()
    conn.close()

    return status



In [3]:
polygon_coords = [
    (102.26539189185458, 16.54356817627857), 
    (101.95606037474194, 16.08328194532515),
    (101.93394678768034, 16.647561388673765), 
    (102.36400115858939, 16.538869127991614), 
    (102.26539189185458, 16.54356817627857)
]

In [4]:
polygon_to_geotiff(polygon_coords)

"File 'output_20240627_065024.tiff' created."